# Analysis of VOO historical price

## Which is the best day to buy the stock?

#### Importing relevant packages

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#### Importing VOO dataset

In [2]:
df = pd.read_csv('VOO.csv', parse_dates = ['Date'])
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.077744,26500
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.445847,8600
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.470108,33750
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.454094,59400
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.662155,9250
...,...,...,...,...,...,...,...
2931,2022-05-02,378.790009,382.269989,372.329987,381.079987,381.079987,10737000
2932,2022-05-03,381.619995,385.059998,380.000000,382.730011,382.730011,5607700
2933,2022-05-04,383.350006,394.929993,380.279999,394.339996,394.339996,7367500
2934,2022-05-05,390.220001,390.309998,376.350006,380.489990,380.489990,10477300


#### Creating Day of Week columns

In [3]:
# Converting Date column into Datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Creating day of week column 
df['DoW'] = df['Date'].dt.day_name()
df

,Date,Open,High,Low,Close,Adj Close,Volume,DoW
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.077744,26500,Thursday
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.445847,8600,Friday
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.470108,33750,Monday
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.454094,59400,Tuesday
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.662155,9250,Wednesday
...,...,...,...,...,...,...,...,...
2931,2022-05-02,378.790009,382.269989,372.329987,381.079987,381.079987,10737000,Monday
2932,2022-05-03,381.619995,385.059998,380.000000,382.730011,382.730011,5607700,Tuesday
2933,2022-05-04,383.350006,394.929993,380.279999,394.339996,394.339996,7367500,Wednesday
2934,2022-05-05,390.220001,390.309998,376.350006,380.489990,380.489990,10477300,Thursday


## Finding the average dip per day 


In [4]:
# Adding an average price column
df['Average_Price'] = (df['High'] + df['Low']) / 2
df

,Date,Open,High,Low,Close,Adj Close,Volume,DoW,Average_Price
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.077744,26500,Thursday,101.819999
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.445847,8600,Friday,101.580002
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.470108,33750,Monday,102.819999
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.454094,59400,Tuesday,102.930000
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.662155,9250,Wednesday,102.890000
...,...,...,...,...,...,...,...,...,...
2931,2022-05-02,378.790009,382.269989,372.329987,381.079987,381.079987,10737000,Monday,377.299988
2932,2022-05-03,381.619995,385.059998,380.000000,382.730011,382.730011,5607700,Tuesday,382.529999
2933,2022-05-04,383.350006,394.929993,380.279999,394.339996,394.339996,7367500,Wednesday,387.604996
2934,2022-05-05,390.220001,390.309998,376.350006,380.489990,380.489990,10477300,Thursday,383.330002


In [5]:
# Subsetting the columns that I will be using
data = df.loc[:, ['DoW', 'Average_Price']]


In [6]:
# Finding the day to day percentage change 
data['perc_change_from_previous_day'] = (data['Average_Price'].diff() / data['Average_Price']) * 100


In [7]:
# Grouping by DoW and finding the mean
data.groupby('DoW')['perc_change_from_previous_day'].mean().reset_index().sort_values(by = 'perc_change_from_previous_day', ascending = True)

,DoW,perc_change_from_previous_day
1,Monday,-0.025508
2,Thursday,0.009236
0,Friday,0.036056
4,Wednesday,0.071124
3,Tuesday,0.107030


#### Accounting for holidays and removing weekends

In [8]:
# Getting all calendar days from dataset
dates = pd.date_range(df['Date'].min(), df['Date'].max())

# Adding it to a dataframe called calendar_dates
calendar_dates = pd.DataFrame({'calendar_dates': dates})

# Getting full calendar days
full_calendar = pd.merge(calendar_dates, df, how = 'left', left_on = 'calendar_dates', right_on = 'Date')

# Regenerating DoW
full_calendar['DoW'] = full_calendar['calendar_dates'].dt.day_name()

# Removing weekends
full_calendar = full_calendar[~full_calendar['DoW'].isin(['Saturday', 'Sunday'])]

calendar_dates


,calendar_dates
0,2010-09-09
1,2010-09-10
2,2010-09-11
3,2010-09-12
4,2010-09-13
...,...
4253,2022-05-02
4254,2022-05-03
4255,2022-05-04
4256,2022-05-05


In [9]:
# Filling up the NAs with the next day's value
full_calendar = full_calendar.fillna(method = 'bfill', axis = 'rows')


In [10]:
# Finding the change in price from previous day
full_calendar['perc_change_from_previous_day'] = (full_calendar['Average_Price'].diff() / full_calendar['Average_Price']) * 100

# Finding the mean of the changes
full_calendar.groupby('DoW')['perc_change_from_previous_day'].mean().reset_index().sort_values(by = 'perc_change_from_previous_day', ascending = True)

,DoW,perc_change_from_previous_day
1,Monday,-0.006403
2,Thursday,0.011376
0,Friday,0.028470
4,Wednesday,0.060836
3,Tuesday,0.102658


Looking at the rates of returns, as a long term investor, Mondays would be best day to invest for me since I am not concerned with day-to-day changes in prices and prices seem to be lowest on Mondays. This also supports the Weekend Effect which is a phenomemon in which stock returns on Mondays are often significantly lower than those of the immediately preceding Friday.

## How much would I have gotten if bought a stock each week since the beginning?

In [11]:
# Creating a function to calculate how much I would have earned if I invested from the start of the VOO stock
# Assumptions: 
# (1) There are no dividends payouts even though VOO pays dividends quarterly
# (2) The amount invested remains the same throughout each time
# (3) There are no platform/transaction fees included each time money is deposited


def get_total(day_of_week, amount_invested):
    temp = full_calendar[full_calendar['DoW'] == day_of_week]
    temp['num_shares'] = amount_invested / temp['Average_Price']
    final_price = temp['Average_Price'].iloc[-1] # to find the final value of my account as of the final date in the data which is 6th of May
    total_amount = temp['num_shares'].sum() * final_price
    return total_amount

get_total('Monday', 700)

844909.0621884665

In [12]:
# Creating a for loop to find the earnings as of 6th May
days = full_calendar['DoW'].unique()
earnings = []

for i in days:
    earning = get_total(i, 700)
    earnings.append(earning)
    print(i, earning)


Thursday 859412.6819491356
Friday 845170.2805928095
Monday 844909.0621884665
Tuesday 855773.2313706353
Wednesday 866580.7187278626
